In [40]:
import pandas as pd

1. Write a function that estimates the emission parameters from the training set using MLE (maximum likelihoodestimation):
Which training set?

file_to_df is a function to convert some data file into a dataframe.

In [41]:
def file_to_df (file_path):


  with open(file_path, 'r',encoding='utf-8') as file:
      lines = file.readlines()
      data = [line.strip().split(' ', maxsplit=2)[:2] for line in lines]

  # Create the dataframe
  df = pd.DataFrame(data, columns=['x', 'y'])
  #drop rows where value for y=None
  df = df.dropna(subset=['y'])

  df['x'] = df['x'].astype(str)
  df['y'] = df['y'].astype(str)
  # Display the dataframe
  #print("Here is dataframe created for the file path: ",file_path,"\n")
  #print(df.to_string)
  return df

file_path_train_es='Data/ES/train'
df_train_es=file_to_df(file_path_train_es)

In [42]:
def count_y(df,y_value):
    unique_counts = df['y'].value_counts().to_dict()
    print("unique counts:",unique_counts)
    return unique_counts[y_value]


In [43]:
def y_star_with_smallest_count(df):
     unique_counts = df['y_star'].value_counts().to_dict()
     min_count = min(unique_counts.values())
     y_with_min_count = [key for key, value in unique_counts.items() if value == min_count]         
     return y_with_min_count[0]         

In [44]:
def create_df_filtered_for_y_value(df,y_value):
  return df[df['y'] == y_value]


In [45]:
def create_df_x_count_y_to_x(df):
    df_x_count_y_to_x = df['x'].value_counts().reset_index()

    df_x_count_y_to_x.columns = ['x', 'count_y_to_x']

    return df_x_count_y_to_x

In [46]:
def create_ls_of_all_y_values(df):
    unique_values = df['y'].unique()
    return (unique_values)
print(create_ls_of_all_y_values(df_train_es))

['O' 'B-positive' 'B-negative' 'B-neutral' 'I-neutral' 'I-positive'
 'I-negative']


In [47]:
def create_df_e_x_y_train(train_df,y_value):
   
    df_train_filtered_for_y = create_df_filtered_for_y_value(train_df,y_value)
    df_e_x_y_train=create_df_x_count_y_to_x(df_train_filtered_for_y)
    df_e_x_y_train['e(x|y)'] = df_e_x_y_train['count_y_to_x']/(count_y(train_df,y_value))
    df_e_x_y_train['y']=y_value
    return df_e_x_y_train


df_e_x_y_train_for_I_neutral=create_df_e_x_y_train(df_train_es,"B-positive")
print(df_e_x_y_train_for_I_neutral)
print(df_e_x_y_train_for_I_neutral['e(x|y)'].sum())

unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
               x  count_y_to_x    e(x|y)           y
0         comida           169  0.145690  B-positive
1       servicio           122  0.105172  B-positive
2    restaurante            46  0.039655  B-positive
3          trato            44  0.037931  B-positive
4       ambiente            33  0.028448  B-positive
..           ...           ...       ...         ...
280       dorada             1  0.000862  B-positive
281      detalle             1  0.000862  B-positive
282   cantidades             1  0.000862  B-positive
283       Atteca             1  0.000862  B-positive
284        menus             1  0.000862  B-positive

[285 rows x 4 columns]
1.0000000000000002


In [48]:
def create_df_e_x_y_test(test_df,y_value,train_df):
    k=1
    #print(test_df.info())
    df_test_filtered_for_y = create_df_filtered_for_y_value(test_df,y_value)
    #print(df_test_filtered_for_y)
    df_e_x_y_test=create_df_x_count_y_to_x(df_test_filtered_for_y)
    #print (df_e_x_y_test)
    
    count_y_plus_k=count_y(test_df,y_value)+k

    #make a list of all the x values in the train_df's x column
   

    
    train_df_x_values = train_df['x'].tolist()
    #print(train_df_x_values)
    
    df_e_x_y_test['x'] = df_e_x_y_test['x'].apply(lambda x: x if x in train_df_x_values else "#UNK#") #if word does or does not appear in training set
    #print(df_e_x_y_test)
    
    df_e_x_y_test['e(x|y)'] = df_e_x_y_test.apply(lambda row: (row['count_y_to_x'] / count_y_plus_k) if row['x'] != '#UNK#' else (k / count_y_plus_k), axis=1)
    print(df_e_x_y_test['e(x|y)'].sum())

    #case 1
    #df_e_x_y_test['e(x|y)'] = df_e_x_y_test['count_y_to_x']/count_y_plus_k

    #case 2
    #df_e_x_y_test['e(x|y)'] = k/count_y_plus_k


    return df_e_x_y_test


file_path_test_es='Data/ES/dev.out'
df_test_es=file_to_df(file_path_test_es)
df_e_x_y_test_for_I_neutral=create_df_e_x_y_test(df_test_es,"B-positive",df_train_es)
print(df_e_x_y_test_for_I_neutral)
# count_unk = 0
# value_counts = df_e_x_y_test_for_I_neutral['x'].value_counts()
# print(value_counts)
# if '#UNK#' in value_counts.index:
#     count_unk = value_counts['#UNK#']
# print(count_unk)

unique counts: {'O': 3998, 'B-positive': 160, 'B-negative': 61, 'I-positive': 49, 'I-negative': 36, 'B-neutral': 8}
0.9937888198757765
               x  count_y_to_x    e(x|y)
0       servicio            24  0.149068
1         comida            23  0.142857
2    restaurante             8  0.049689
3       ambiente             7  0.043478
4         platos             6  0.037267
..           ...           ...       ...
67      cocinero             1  0.006211
68        tartar             1  0.006211
69       postres             1  0.006211
70  localización             1  0.006211
71         comer             1  0.006211

[72 rows x 3 columns]


In [49]:


def create_e_x_y_df_train_all_y_values(file_path):
    
  df_train=file_to_df(file_path)
  #print(df_train_es)
  ls_df_train=[]
  ls_y_values=create_ls_of_all_y_values(df_train)
  #print(len(ls_y_values))
  for y_value in ls_y_values:
    if y_value!=None:
      df_e_x_y=create_df_e_x_y_train(df_train,y_value)
      ls_df_train.append(df_e_x_y)
  #print(len(ls_df_train))
  
  combined_df_train=pd.concat(ls_df_train,axis=0)
  return (combined_df_train)
#file_path_train_es='Data/ES/train'
#print(create_e_x_y_df_train_all_y_values(file_path_train_es))

In [50]:
def create_df_x_to_y_star(file_path):
    e_x_y_df_train=create_e_x_y_df_train_all_y_values(file_path)
    
    # Group by 'x' and find the maximum 'e(x|y)' value for each group
    df_x_to_y_star = e_x_y_df_train.groupby('x')['e(x|y)'].max().reset_index()

    # Find the corresponding 'y' values for the maximum 'e(x|y)' values
    df_x_to_y_star = pd.merge(df_x_to_y_star, e_x_y_df_train, on=['x', 'e(x|y)'])

    # Rename the columns
    df_x_to_y_star.columns = ['x', 'max_e(x|y)','count_y_to_x', 'y_star']

    # Print the resulting DataFrame
    return (df_x_to_y_star)
    
file_path_train_es='Data/ES/train'    
df_x_to_y_star=create_df_x_to_y_star(file_path_train_es)
csv_x_to_y_star = df_x_to_y_star.to_csv('train_x_to_y_star.csv', index = True)

unique counts:

 {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}


In [51]:
def generate_y_values(file_path_dev_in,file_path_train,file_path_dev_p1_out):
    df_train=create_df_x_to_y_star(file_path_train)
    x_values=df_train['x'].tolist()
    #the y_label that has the smallest count_y will generate the biggest k / count_y_plus_k which will give the maximum e(x|y) which means that y_label is y_star
    #so that y_label is:
    y_label=y_star_with_smallest_count(df_train) 
    with open(file_path_dev_in, 'r',encoding='utf-8') as file:
      lines = file.readlines()
      for l in range(len(lines)):
        line=lines[l].strip()
        if line in x_values:
          possible_y_values=df_train[df_train['x'] == line]['y_star'].tolist()
          lines[l]=line+" "+possible_y_values[0]
         
          if (len(possible_y_values)!=1):
            print ("something wrong: x_vlaues in df_train not unique for some reason,for line: ",l)
        else:
          if (line!=""):
             line=line+" "+y_label
             lines[l]=line
          else:
            lines[l]=""
          
    with open(file_path_dev_p1_out, 'w',encoding='utf-8') as file:
        
        for line in lines:
          file.write(line+"\n")   
         
         

   
file_path_dev_in_es = 'Data/ES/dev.in'   
file_path_train_es='Data/ES/train'  
file_path_dev_p1_out_es='Data/ES/dev.p1.out' 
generate_y_values(file_path_dev_in_es,file_path_train_es,file_path_dev_p1_out_es)



unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}
unique counts: {'O': 29035, 'B-positive': 1160, 'B-negative': 381, 'I-positive': 314, 'I-negative': 171, 'B-neutral': 72, 'I-neutral': 43}


In [52]:
file_path_dev_p1_out_es='Data/ES/dev.p1.out'
file_dev_out_es='Data/ES/dev.out'
df_dev_p1_out_es= file_to_df (file_path_dev_p1_out_es)
df_dev_out_es=file_to_df (file_dev_out_es)
print(df_dev_p1_out_es.shape[0])
print(df_dev_out_es.shape[0])

df1=df_dev_p1_out_es
df2=df_dev_out_es

# Compare rows
comparison = df1.equals(df2)

# Get the count of rows that are the same
count_same = df1[df1.eq(df2).all(axis=1)].shape[0]

# Get the count of rows that are not the same
count_not_same = df1[df1.ne(df2).any(axis=1)].shape[0]

# Get the rows that are not the same
rows_not_same = df1[~df1.eq(df2).all(axis=1)]

total_number_of_correctly_predicted_entries=count_same
total_number_of_predicted_entities=df1.shape[0]
precision=total_number_of_correctly_predicted_entries/total_number_of_predicted_entities
print(precision)

# Display the results
# print("Comparison result:", comparison)
# print("Count of rows that are the same:", count_same)
# print("Count of rows that are not the same:", count_not_same)
# print("Rows that are not the same:\n", rows_not_same)

1
4312
0.0


C:\Users\Sherin Saji\AppData\Local\Temp\ipykernel_9528\2950593151.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  count_same = df1[df1.eq(df2).all(axis=1)].shape[0]
C:\Users\Sherin Saji\AppData\Local\Temp\ipykernel_9528\2950593151.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  count_not_same = df1[df1.ne(df2).any(axis=1)].shape[0]
C:\Users\Sherin Saji\AppData\Local\Temp\ipykernel_9528\2950593151.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rows_not_same = df1[~df1.eq(df2).all(axis=1)]
